In [3]:
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

class TextGenerator:
    def __init__(self, model_name, device="cuda:0"):
        self.model = AutoGPTQForCausalLM.from_quantized(model_name, device=device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        self.device = device
    
    def generate_text(self, input_prompt):
        input_ids = self.tokenizer([f'<s>Human: {input_prompt}\n</s><s>Assistant: '], return_tensors="pt", add_special_tokens=False).input_ids.to(self.device)
        
        generate_input = {
            "input_ids": input_ids,
            "max_new_tokens": 512,
            "do_sample": True,
            "top_k": 50,
            "top_p": 0.95,
            "temperature": 0.3,
            "repetition_penalty": 1.3,
            "eos_token_id": self.tokenizer.eos_token_id,
            "bos_token_id": self.tokenizer.bos_token_id,
            "pad_token_id": self.tokenizer.pad_token_id
        }
        
        generate_ids = self.model.generate(**generate_input)
        generated_text = self.tokenizer.decode(generate_ids[0])
        return generated_text

# Create an instance of the TextGenerator class
model_name = "FlagAlpha/Llama2-Chinese-13b-Chat-4bit"
text_generator = TextGenerator(model_name)

# Generate text using the class method
input_prompt = "产业创新指标如何量化"
generated_text = text_generator.generate_text(input_prompt)
print(generated_text)


skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><unk> : 产业创新指标如何量化
</s><s>Assistant: 1. 专利数：探究公司获得的各类专利数，包括国际和本地专利。这是一个重要的度量标准，因为通过发明来保持竞争力可以提高质量水平并确定其在市场中的位置。
2. 研发成果表现：了解公司实际上能否将技术转换到商品或服务之间，同时也需要关注该公司所处行业内相对应用程序的比例。
3. 知识价值分配：查看员工、客户等人们对于特有知识的使用情形，而不只是限制自身收入。
4. 版图展望：判断公司与其他参与者（如合作伙伴）共同开发出未来计划中的项目，以及这些项目的可视性。
5. 社会影响：调查公司从事的活动对社区进行改变方面的反应，如支付给当地组织、基金会等。
6. 生态效益：测试公司如何在长期运营下保持健康的经济模式，以及如何控制外部因素对此造成的影响。
7. 文化建设：了解公司的管理思想、传统文化和操作流程，以及如何引导员工认真学习和接受公司的文化。
8. 教育机构联系：了解公司与大型科技企业、政府机构、非营利组织等结交，以加速创新进程。
9. 法律安全：了解公司如何保护智慧財産权，以及如何应对


### 信息提取能力

In [5]:
input_prompt = """
产业分类：
在经济研究和经济管理中，经常使用的分类方法主要有两大领域、两大部类分类法，三次产业分类法，资源密集度分类法和国际标准产业分类。
两大领域、两大部类分类法。这种分类法就是按生产活动的性质及其产品属性对产业进行分类。按生产活动性质，把产业部门分为物质资料生产部门和非物质资料生产部门两大领域，
前者指从事物质资料生产并创造物质产品的部门，包括农业、工业、建筑业、运输邮电业、商业等；后者指不从事物质资料生产而只提供非物质性服务的部门，包括科学、文化、教育、新闻、卫生、金融、保险、物业、咨询等部门。
总结一下产业分类两大领域是什么
"""
generated_text = text_generator.generate_text(input_prompt)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><unk> : 
产业分类：
在经济研究和经济管理中，经常使用的分类方法主要有两大领域、两大部类分类法，三次产业分类法，资源密集度分类法和国际标准产业分类。
两大领域、两大部类分类法。这种分类法就是按生产活动的性质及其产品属性对产业进行分类。按生产活动性质，把产业部门分为物质资料生产部门和非物质资料生产部门两大领域，
前者指从事物质资料生产并创造物质产品的部门，包括农业、工业、建筑业、运输邮电业、商业等；后者指不从事物质资料生产而只提供非物质性服务的部门，包括科学、文化、教育、新闻、卫生、金融、保险、物业、咨询等部门。
总结一下产业分类两大领域是什么

</s><s> Assistant: 根据生产活动的特点以及所制造或提供的产品属性来将企业或组織按照二元分类，可以将之分成“物质资料生产”与“非物质资料生产”两个领域。物质资料生产则包括农业、工业、建筑业、交通运输、商业等相关行业，而非物质资料生产则包括科技、文化、教育、新闻传播、健康保健、金融、保險、房地产等相关行业。
</s>


In [19]:
input_prompt = """
党的十八大以来，以习近平同志为核心的党中央高度重视资本市场工作，加强对资本市场的集中统一领导，作出一系列重大决策部署，明确提出要通过深化改革，打造一个规范、透明、开放、有活力、有韧性的资本市场。习近平总书记深刻指出，要把发展直接融资放在重要位置，形成融资功能完备、基础制度扎实、市场监管有效、投资者合法权益得到有效保护的多层次资本市场体系；要把主动防范化解系统性金融风险放在更加重要的位置，科学防范，早识别、早预警、早发现、早处置，着力防范化解重点领域风险，着力完善金融安全防线和风险应急处置机制；等等。这些重要论述为新时代资本市场改革发展指明了方向。特别是2017年全国金融工作会议以来，在以习近平同志为核心的党中央坚强领导下，我国紧扣金融供给侧结构性改革的主线，坚持用改革的思路和办法来破解资本市场体制机制性障碍，坚定推进全面深化资本市场改革，推动资本市场发生深刻的结构性变化，服务经济发展实现量质双升。
股债融资稳步增长。2017年至2021年，首次公开发行股票（IPO）和再融资金额合计5.2万亿元，交易所债券市场发行33.9万亿元。其中，2021年IPO和再融资金额合计约1.5万亿元，股票和交易所债券市场融资合计超10万亿元，均创历史新高。
市场结构明显优化。截至2022年6月末，A股战略性新兴行业上市公司超过2200家，新一代信息技术、生物医药等高科技行业市值占比由2017年初的约20%增长至约37%，上市公司研发投入占全国企业研发支出的一半以上，上市公司作为实体经济“基本盘”、转型发展“领跑者”的角色更加凸显。投资者结构逐步改善，境内专业机构投资者和外资持仓占流通市值比重由2017年初的15.8%提升至23.5%。
多层次市场体系不断健全。科创板、创业板试点注册制相继成功落地，大大提升了资本市场对优质企业的吸引力。深化新三板改革、设立北京证券交易所，打造服务创新型中小企业主阵地迈出关键一步。《中华人民共和国期货和衍生品法》审议通过，有效填补了期货和衍生品领域的“基本法”空白。截至2022年6月末，期货期权品种已有94个，基本涵盖国计民生主要领域，在2021年以来国际大宗商品价格大幅上涨的背景下，我国动力煤、铁矿石等主要期货品种价格及涨幅均小于现货、小于境外，为保供稳价大局贡献了积极力量。
优化资源配置的功能进一步发挥。资本市场基础制度的适应性包容性明显提升，市场化的激励约束机制不断完善。资本市场并购重组主渠道作用不断强化，近5年并购重组交易金额约10万亿元，激发了市场主体活力。退市制度进一步健全，今年已有42家公司强制退市，进退有序、优胜劣汰的市场生态正加速形成。
高水平制度型开放稳步推进。统筹开放和安全，推动市场、产品和机构全方位开放。证券基金期货行业外资股比限制全面放开。互联互通不断深化，沪深港通制度安排持续优化，交易型开放式基金（ETF）纳入沪深港通标的落地实施，沪伦通机制对内拓展到深交所，对外拓展至德国、瑞士。A股纳入国际知名指数并不断提升比重，在香港推出A股指数期货。外资连续多年保持净流入，我国资本市场的国际吸引力和竞争力明显增强。

新一代信息技术、生物医药等高科技行业市值占比多少?
"""
generated_text = text_generator.generate_text(input_prompt)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><unk> : 
党的十八大以来，以习近平同志为核心的党中央高度重视资本市场工作，加强对资本市场的集中统一领导，作出一系列重大决策部署，明确提出要通过深化改革，打造一个规范、透明、开放、有活力、有韧性的资本市场。习近平总书记深刻指出，要把发展直接融资放在重要位置，形成融资功能完备、基础制度扎实、市场监管有效、投资者合法权益得到有效保护的多层次资本市场体系；要把主动防范化解系统性金融风险放在更加重要的位置，科学防范，早识别、早预警、早发现、早处置，着力防范化解重点领域风险，着力完善金融安全防线和风险应急处置机制；等等。这些重要论述为新时代资本市场改革发展指明了方向。特别是2017年全国金融工作会议以来，在以习近平同志为核心的党中央坚强领导下，我国紧扣金融供给侧结构性改革的主线，坚持用改革的思路和办法来破解资本市场体制机制性障碍，坚定推进全面深化资本市场改革，推动资本市场发生深刻的结构性变化，服务经济发展实现量质双升。
股债融资稳步增长。2017年至2021年，首次公开发行股票（IPO）和再融资金额合计5.2万亿元，交易所债券市场发行33.9万亿元。其中，2021年IPO和再融资金额合计约1.5万亿元，股票和交易所债券市场融资合计超10万亿元，均创历史新高。
市场结构明显优化。截至2022年6月末，A股战略性新兴行业上市公司超过2200家，新一代信息技术、生物医药等高科技行业市值占比由2017年初的约20%增长至约37%，上市公司研发投入占全国企业研发支出的一半以上，上市公司作为实体经济“基本盘”、转型发展“领跑者”的角色更加凸显。投资者结构逐步改善，境内专业机构投资者和外资持仓占流通市值比重由2017年初的15.8%提升至23.5%。
多层次市场体系不断健全。科创板、创业板试点注册制相继成功落地，大大提升了资本市场对优质企业的吸引力。深化新三板改革、设立北京证券交易所，打造服务创新型中小企业主阵地迈出关键一步。《中华人民共和国期货和衍生品法》审议通过，有效填补了期货和衍生品领域的“基本法”空白。截至2022年6月末，期货期权品种已有94个，基本涵盖国计民生主要领域，在2021年以来国际大宗商品价格大幅上涨的背景下，我国动力煤、铁矿石等主要期货品种价格及涨幅均小于现货、小于境外，为保供稳价大局贡献了积极力量。
优化资源配置的功能进一步发挥。资本市场基础制度的适应性